In [ ]:
# !pip install chromadb sentence-transformers transformers llama_index
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
import json
from pathlib import Path
from llama_index import download_loader

JSONReader = download_loader("JSONReader")

loader = JSONReader()
documents = loader.load_data(Path('./data/courses.json'))

In [ ]:
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-base-en")

In [ ]:
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/openchat_3.5-GGUF/resolve/main/openchat_3.5.Q5_K_M.gguf",
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    context_window=8192,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

In [ ]:
from llama_index import ServiceContext, set_global_service_context

service_context = ServiceContext.from_defaults(
    chunk_size=128,
    chunk_overlap=30,
    llm=llm,
    embed_model=embed_model,
    )

set_global_service_context(service_context)

In [ ]:
import chromadb
from llama_index import VectorStoreIndex
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext

# Initialze chromadb
db = chromadb.PersistentClient(path="./chroma.db")
chroma_collection = db.get_or_create_collection("courses_collection")

# create a vector store
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create a vector index
vector_index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

In [ ]:
from llama_index import KnowledgeGraphIndex
from llama_index.graph_stores import SimpleGraphStore
from llama_index.storage.storage_context import StorageContext

# create a graph store
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

# create a knowledge graph index
kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    include_embeddings=True,
)

In [ ]:
from typing import List

from llama_index import QueryBundle
from llama_index.schema import NodeWithScore
from llama_index.retrievers import BaseRetriever, VectorIndexRetriever, KGTableRetriever


class VectorPlusKGRetreiver(BaseRetriever):
    """A custom retriever that performs both vector search and knowledge Graph search"""

    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        kg_retriever: KGTableRetriever,
        mode: str = "OR",
    ) -> None:
        """Init params."""

        self._vector_retriever = vector_retriever
        self._kg_retriever = kg_retriever
        if mode not in ("AND", "OR"):
            raise ValueError("Invalid mode.")
        self._mode = mode
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve nodes given query."""

        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        kg_nodes = self._kg_retriever.retrieve(query_bundle)

        vector_ids = {n.node.node_id for n in vector_nodes}
        kg_ids = {n.node.node_id for n in kg_nodes}

        combined_dict = {n.node.node_id: n for n in vector_nodes}
        combined_dict.update({n.node.node_id: n for n in kg_nodes})

        if self._mode == "AND":
            retrieve_ids = vector_ids.intersection(kg_ids)
        else:
            retrieve_ids = vector_ids.union(kg_ids)

        retrieve_nodes = [combined_dict[rid] for rid in retrieve_ids]
        return retrieve_nodes

In [ ]:
# create retreivers
vector_retriever = VectorIndexRetriever(index=vector_index)
kg_retriever = KGTableRetriever(
    index=kg_index, retriever_mode="keyword", include_text=False
)
retriever = VectorPlusKGRetreiver(vector_retriever, kg_retriever)

In [ ]:
from llama_index import get_response_synthesizer
from llama_index.query_engine import RetrieverQueryEngine

# create response synthesizer
response_synthesizer = get_response_synthesizer(
    service_context=service_context,
    response_mode="compact",
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
res = query_engine.query("What is the code of the fluid mechanics course?")
print(res)

In [ ]:
with open("./data/questions.json") as f:
    questions = json.load(f)

In [ ]:
answers = []
for question in questions:
    answers.append({"question": question, "answer": str(query_engine.query(question))})

In [ ]:
with open("./results/<MODEL-NAME>.json", "w") as f:
    json.dump(answers, f)